# Parameter Management

The ultimate goal of training deep networks is to find good parameter values for a given architecture. When everything is standard, the __torch.nn.Sequential__ class is a perfectly good tool for it. However, very few models are entirely standard and most scientists want to build things that are novel. This section shows how to manipulate parameters. In particular we will cover the following aspects:

- Accessing parameters for debugging, diagnostics, to visualize them or to save them is the first step to understanding how to work with custom models.
- Secondly, we want to set them in specific ways, e.g. for initialization purposes. We discuss the structure of parameter initializers.
- Lastly, we show how this knowledge can be put to good use by building networks that share some parameters.

As always, we start from our trusty Multilayer Perceptron with a hidden layer. This will serve as our choice for demonstrating the various features.

In [1]:
import torch
import torch.nn as nn

net = nn.Sequential()
net.add_module('Linear_1', nn.Linear(20, 256, bias = False))
net.add_module('relu', nn.ReLU())
net.add_module('Linear_2', nn.Linear(256, 10, bias = False))

# the init_weights function initializes the weights of our multi-layer perceptron 

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)

# the net.apply() applies the above stated initialization of weights to our net        

net.apply(init_weights) 

x = torch.randn(2,20)   #initialing a random tensor of shape (2,20)
net(x)  #Forward computation


C:\Users\Akshit Mittal\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  del sys.path[0]


tensor([[ 0.6711,  0.4682,  0.1206, -0.1714,  0.6909,  0.0103, -0.3668,  0.2536,
         -0.2216,  0.2312],
        [ 0.1162, -0.1884,  0.0085, -0.4268, -0.0854, -0.1953, -0.4664, -0.3036,
         -0.2258,  0.0847]], grad_fn=<MmBackward>)

## Parameter Access


In the case of a Sequential class we can access the parameters with ease, simply by calling __net.parameters__. Let’s try this out in practice by inspecting the parameters.

In [2]:
net.parameters

<bound method Module.parameters of Sequential(
  (Linear_1): Linear(in_features=20, out_features=256, bias=False)
  (relu): ReLU()
  (Linear_2): Linear(in_features=256, out_features=10, bias=False)
)>

The output tells us a number of things. Firstly, there are 3 layers; 2 linear layers and 1 ReLU layer as we would expect. The output also specifies the shapes that we would expect from linear layers. In particular the names of the parameters are very useful since they allow us to identify parameters uniquely even in a network of hundreds of layers and with nontrivial structure. Also, output also tells us that bias is __False__ as we specified it.  


### Targeted Parameters

In order to do something useful with the parameters we need to access them, though. There are several ways to do this, ranging from simple to general. Let’s look at some of them.

In [3]:
print(net[0].bias)

None


It returns the bias of the first linear layer. Since we initialized the bias to be __False__, the output is None. We can also access the parameters by name, such as _**Linear_1**_. Both methods are entirely equivalent but the first method leads to much more readable code.

In [4]:
print(net.Linear_1.weight)
print(net[0].weight)

Parameter containing:
tensor([[-0.0538,  0.0277, -0.0919,  ...,  0.1462,  0.0200,  0.0782],
        [-0.0448,  0.0143,  0.1370,  ...,  0.1414,  0.0789,  0.0143],
        [ 0.1147,  0.1246, -0.0902,  ..., -0.1053, -0.1316,  0.0570],
        ...,
        [ 0.0820, -0.0803,  0.0340,  ..., -0.0170,  0.1419, -0.0437],
        [-0.0375,  0.0196, -0.0634,  ..., -0.0050,  0.1458,  0.0978],
        [ 0.0128,  0.0569,  0.0877,  ...,  0.0289,  0.1298,  0.0806]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0538,  0.0277, -0.0919,  ...,  0.1462,  0.0200,  0.0782],
        [-0.0448,  0.0143,  0.1370,  ...,  0.1414,  0.0789,  0.0143],
        [ 0.1147,  0.1246, -0.0902,  ..., -0.1053, -0.1316,  0.0570],
        ...,
        [ 0.0820, -0.0803,  0.0340,  ..., -0.0170,  0.1419, -0.0437],
        [-0.0375,  0.0196, -0.0634,  ..., -0.0050,  0.1458,  0.0978],
        [ 0.0128,  0.0569,  0.0877,  ...,  0.0289,  0.1298,  0.0806]],
       requires_grad=True)


Note that the weights are nonzero. This is by design since we applied __Xavier initialization__ to our network. We can also compute the gradient with respect to the parameters. It has the same shape as the weight. However, since we did not invoke backpropagation yet, the output is None.

In [5]:
print(net[0].weight.grad)

None


### All parameters at once

Accessing parameters as described above can be a bit tedious, in particular if we have more complex blocks, or blocks of blocks (or even blocks of blocks of blocks), since we need to walk through the entire tree in reverse order to how the blocks were constructed. To avoid this, blocks come with a method __state_dict__ which grabs all parameters of a network in one dictionary such that we can traverse it with ease. It does so by iterating over all constituents of a block and calls __state_dict__ on subblocks as needed. To see the difference consider the following:

In [6]:
print(net[0].state_dict) # parameters only for first layer
print(net.state_dict) # parameters for entire network

<bound method Module.state_dict of Linear(in_features=20, out_features=256, bias=False)>
<bound method Module.state_dict of Sequential(
  (Linear_1): Linear(in_features=20, out_features=256, bias=False)
  (relu): ReLU()
  (Linear_2): Linear(in_features=256, out_features=10, bias=False)
)>


This provides us with a third way of accessing the parameters of the network. If we wanted to get the value of the weight term of the second linear layer we could simply use this:

In [7]:
net.state_dict()['Linear_1.weight']

tensor([[-0.0538,  0.0277, -0.0919,  ...,  0.1462,  0.0200,  0.0782],
        [-0.0448,  0.0143,  0.1370,  ...,  0.1414,  0.0789,  0.0143],
        [ 0.1147,  0.1246, -0.0902,  ..., -0.1053, -0.1316,  0.0570],
        ...,
        [ 0.0820, -0.0803,  0.0340,  ..., -0.0170,  0.1419, -0.0437],
        [-0.0375,  0.0196, -0.0634,  ..., -0.0050,  0.1458,  0.0978],
        [ 0.0128,  0.0569,  0.0877,  ...,  0.0289,  0.1298,  0.0806]])

### Rube Goldberg strikes again

Let's see how the parameter naming conventions work if we nest multiple blocks inside each other. For that we first define a function that produces blocks (a block factory, so to speak) and then we combine these inside yet larger blocks.

In [8]:
def block1():
    net = nn.Sequential(nn.Linear(16, 32),
                        nn.ReLU(),
                        nn.Linear(32, 16),
                        nn.ReLU())
    return net

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module('block'+str(i), block1())
    return net    
        
rgnet = nn.Sequential()
rgnet.add_module('model',block2())
rgnet.add_module('Last_linear_layer', nn.Linear(16,10))
rgnet.apply(init_weights)
x = torch.randn(2,16)
rgnet(x) # forward computation
    

C:\Users\Akshit Mittal\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  del sys.path[0]


tensor([[ 0.2980, -0.2109, -0.0334,  0.1267,  0.1291, -0.3061, -0.0142,  0.1217,
          0.0752, -0.0719],
        [ 0.3117, -0.2294, -0.0615,  0.1087,  0.1479, -0.2471, -0.0197,  0.1489,
          0.0400, -0.0269]], grad_fn=<ThAddmmBackward>)

Now that we are done designing the network, let's see how it is organized. __state_dict__ provides us with this information, both in terms of naming and in terms of logical structure.

In [9]:
print(rgnet.state_dict)
print(rgnet.state_dict())

<bound method Module.state_dict of Sequential(
  (model): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=16, bias=True)
      (3): ReLU()
    )
  )
  (Last_linear_layer): Linear(in_features=16, out_features=10, bias=True)
)>
OrderedDict([('model.block0.0.weight', tensor([[-0.2053,  0.1735,  0.2060, -0.083

Since the layers are hierarchically generated, we can also access them accordingly. For instance, to access the first major block, within it the second subblock and then within it, in turn the bias of the first layer, we perform the following.

In [10]:
rgnet[0][1][0].bias.data

tensor([-0.1081, -0.1655,  0.1439, -0.0012, -0.1704, -0.2026,  0.0021, -0.1666,
        -0.0365, -0.0055,  0.1888, -0.1573,  0.1651, -0.1471, -0.0822,  0.1610,
         0.0754,  0.0066, -0.0257,  0.2010, -0.0265, -0.0033,  0.0629, -0.0766,
        -0.1390, -0.0753, -0.0628,  0.2232,  0.0385, -0.2445, -0.0302,  0.0114])

## Parameter Initialization

To initialize the weights of a single layer, we use a function from __torch.nn.init__ . For instance:

In [11]:
linear1 = nn.Linear(2,5,bias=True)
torch.nn.init.normal_(linear1.weight, mean=0, std =0.01)  

Parameter containing:
tensor([[-0.0013, -0.0008],
        [ 0.0133,  0.0068],
        [ 0.0137, -0.0063],
        [ 0.0058,  0.0006],
        [-0.0147,  0.0046]], requires_grad=True)

If we wanted to initialize all parameters to 1, we could do this simply by changing the initializer to Constant(1)

In [12]:
def init_weight(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight)
        
net = nn.Sequential()
net.add_module('Linear_1', nn.Linear(2, 5, bias = False))
net.add_module('Linear_2', nn.Linear(5, 5, bias = False))

net.apply(init_weight)
print(net.state_dict())

OrderedDict([('Linear_1.weight', tensor([[ 0.0860, -0.4133],
        [-0.4154, -1.2724],
        [ 0.7713, -2.3440],
        [-0.8033, -0.3308],
        [ 0.3841,  0.2840]])), ('Linear_2.weight', tensor([[ 0.3219,  0.7539, -0.3076,  0.9368,  0.7773],
        [ 0.0464, -0.2074, -1.1464, -0.5490, -0.0038],
        [-0.1218,  0.6165, -0.0134, -0.4004, -0.6874],
        [ 1.2538,  0.3477, -0.5232, -0.2392, -0.2431],
        [-0.1146,  0.6870,  0.4552, -2.1838,  0.3568]]))])


### Built-in Initialization

Let’s begin with the built-in initializers. The code below initializes all parameters with Gaussian random variables.

In [13]:
def gaussian_normal(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight)
        
net.apply(gaussian_normal)
print(net[0].weight)

Parameter containing:
tensor([[-1.0308, -0.0032],
        [ 1.1529,  0.4452],
        [-1.0855, -0.8679],
        [ 0.1205, -0.8966],
        [ 1.0436,  1.8051]], requires_grad=True)


If we wanted to initialize all parameters to 1, we could do this simply by changing the initializer to __torch.nn.init.constant_(tensor,1)__.

In [14]:
def ones(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 1)
        
net.apply(ones)
print(net.state_dict())



OrderedDict([('Linear_1.weight', tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])), ('Linear_2.weight', tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]))])


If we want to initialize only a specific parameter in a different manner, we can simply set the initializer only for the appropriate subblock (or parameter) for that matter. For instance, below we initialize the __second layer__ to a constant value of __42__ and we use the __Xavier initializer__ for the weights of the __first layer__.

In [15]:
block1 = nn.Sequential()
block1.add_module('Linear_1', nn.Linear(2,5,bias=False))
block2 = nn.Sequential()
block2.add_module('Linear_2', nn.Linear(5,5,bias=False))

model = nn.Sequential()
model.add_module('first', block1)
model.add_module('second', block2)

def xavier_normal(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)
              
block1.apply(xavier_normal)
block2.apply(init_42)
print(model.state_dict())

OrderedDict([('first.Linear_1.weight', tensor([[ 0.6575,  0.6740],
        [ 0.0766, -0.4489],
        [-0.3454, -0.3206],
        [ 0.8269,  0.7122],
        [-0.8657, -0.6929]])), ('second.Linear_2.weight', tensor([[42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.]]))])


C:\Users\Akshit Mittal\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if sys.path[0] == '':


### Custom Initialization

Sometimes, the initialization methods we need are not provided in the init module. At this point, we can implement our desired implementation by writing the desired functions and use them to initialize the weights. In the example below, we pick a decidedly bizarre and nontrivial distribution, just to prove the point. We draw the coefficients from the following distribution:

$$ \begin{aligned} w \sim \begin{cases} U[5, 10] & \text{ with probability } \frac{1}{4} \   
0 & \text{ with probability } \frac{1}{2} \      
U[-10, -5] & \text{ with probability } \frac{1}{4} \end{cases} \end{aligned} $$

In [16]:
def custom(m):
    torch.nn.init.uniform_(m[0].weight, -10,10)
    for i in range(m[0].weight.data.shape[0]):
        for j in range(m[0].weight.data.shape[1]):
            if m[0].weight.data[i][j]<=5 and m[0].weight.data[i][j]>=-5:
                m[0].weight.data[i][j]=0
    
    
m = nn.Sequential(nn.Linear(5,5,bias=False))
custom(m)
print(m.state_dict())
    

OrderedDict([('0.weight', tensor([[-6.6177,  0.0000,  6.2309,  5.1063,  0.0000],
        [-5.1540,  0.0000,  0.0000,  7.2982,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -5.6190, -6.4968],
        [ 0.0000,  5.8972,  8.2195,  0.0000, -6.8732],
        [ 0.0000,  7.2649, -8.0269, -9.7509,  0.0000]]))])


If even this functionality is insufficient, we can set parameters directly. Since __.data__ returns a Tensor we can access it just like any other matrix.

In [17]:
m[0].weight.data +=1
m[0].weight.data[0][0] = 42
m[0].weight.data

tensor([[42.0000,  1.0000,  7.2309,  6.1063,  1.0000],
        [-4.1540,  1.0000,  1.0000,  8.2982,  1.0000],
        [ 1.0000,  1.0000,  1.0000, -4.6190, -5.4968],
        [ 1.0000,  6.8972,  9.2195,  1.0000, -5.8732],
        [ 1.0000,  8.2649, -7.0269, -8.7509,  1.0000]])

## Tied Parameters

In some cases, we want to share model parameters across multiple layers. For instance when we want to find good word embeddings we may decide to use the same parameters both for encoding and decoding of words. Let’s see how to do this a bit more elegantly. In the following we allocate a linear layer and then use it multiple times for sharing the weights.

In [23]:


# We need to give the shared layer a name such that we can reference its
# parameters

shared = nn.Sequential()
shared.add_module('linear_shared', nn.Linear(8,8,bias=False))
shared.add_module('relu_shared', nn.ReLU())                  
net = nn.Sequential(nn.Linear(20,8,bias=False),
               nn.ReLU(),
               shared,
               shared,
               nn.Linear(8,10,bias=False))

net.apply(init_weights)

print(net[2][0].weight==net[3][0].weight)


tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)


C:\Users\Akshit Mittal\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  del sys.path[0]


The above example shows that the parameters of the second and third layer are tied. They are identical rather than just being equal. That is, by changing one of the parameters the other one changes, too. 